In [7]:
import pandas as pd
import json
from unidecode import unidecode

In [56]:
stats_joueurs = pd.read_csv("/Users/bpugnaire/Dev/github/handball-scraping/data/D2H/stats_joueurs_2018-19.csv")


In [16]:
def last_capital_letter(word):
    for i in range(len(word)-1, -1,-1):
        if (word[i].isupper()) :
            return i
    return 0

In [17]:
def first_digit(word):
    for i in range(len(word)-1):
        if (word[i].isdigit()) and not (word[i+1].isdigit()):
            return i+1
    return 0

In [112]:
def upperify_accents(word):
    word = list(word)
    for i in range(1,len(word)-1):
        if not word[i].isupper() and (word[i-1].isupper() and (word[i+1].isupper() or word[i+1]==' ')):
            word[i] = word[i].upper()
    return "".join(word)

In [113]:
stats_joueurs.head(20)

,Unnamed: 0,joueurs,total buts,% total,total buts / mj,buts tirs,% tirs,tirs buts / mj,buts penalty,% penalty,penalty buts / mj,mj
0,0,1 DROUHIN Matthieu,204 / 261,"78,16 %",785,87 / 126,"69,05 %",335,117 / 135,"86,67 %",450,26
1,1,2 BRKLJACIC Luka,184 / 319,"57,68 %",657,102 / 215,"47,44 %",364,82 / 104,"78,85 %",293,28
2,2,3 CARDINAL Youenn,171 / 258,"66,28 %",658,114 / 188,"60,64 %",438,56 / 70,"80,00 %",215,26
3,3,4 NAUDIN Pierrick,167 / 265,"63,02 %",619,118 / 199,"59,30 %",437,50 / 66,"75,76 %",185,27
4,4,5 ONUFRIYENKO Sergiy,162 / 255,"63,53 %",623,136 / 220,"61,82 %",523,26 / 35,"74,29 %",100,26
5,5,6 SISSOKO Boïba,161 / 236,"68,22 %",619,120 / 178,"67,42 %",462,41 / 58,"70,69 %",158,26
6,6,7 BOé Corentin,161 / 282,"57,09 %",644,107 / 210,"50,95 %",428,54 / 72,"75,00 %",216,25
7,7,8 KOLLE Romuald,150 / 246,"60,98 %",625,150 / 246,"60,98 %",625,0 / 0,"0,00 %",0,24
8,8,9 BORRAGAN Javier,142 / 248,"57,26 %",507,122 / 220,"55,45 %",436,20 / 28,"71,43 %",71,28
9,9,10 DUCREUX Yann,141 / 197,"71,57 %",504,102 / 144,"70,83 %",364,39 / 53,"73,58 %",139,28


In [114]:
import re

test_str = 'BRANCA FREITAS'

test_str[next(re.finditer('([A-Z]+[a-z])', 'BRANCA FREITAS')).start(0):]

StopIteration: 

In [115]:
def reformate_stats_joueurs(row):
    joueur = row['joueurs']
    joueur = upperify_accents(joueur)
    try:
        debut_prenom = next(re.finditer('([A-ZÀ-Ú][a-zá-ú])', joueur)).start(0)
        prenom = unidecode(joueur[debut_prenom:])
        nom = unidecode(joueur[first_digit(joueur):debut_prenom-1]).upper()
    except:
        prenom = ''
        nom = ''
    
    matchs = row['mj']
    #total arrets
    x = row["total  buts"].split('/') 
    but_total = x[0].strip()
    occasion_total = x[1].strip()
    pourcentage_buts = row['%  total'].split(' ')[0].strip().replace(',','.')
    total_but_match = str(row['total  buts / mj']/100)
    #arrets sur tir
    x = row['buts  tirs'].split('/')
    arrets_tirs = x[0].strip()
    tirs_total = x[1].strip()
    pourcentage_arrets_tirs = row['%  tirs'].split(' ')[0].strip().replace(',','.')
    total_arret_tirs_match = str(row['tirs  buts / mj']/100)
    #arrets sur penalty
    x = row['buts  penalty'].split('/')
    arrets_penalty = x[0].strip()
    penalty_total = x[1].strip()
    pourcentage_arrets_penalty = row['%  penalty'].split(' ')[0].strip().replace(',','.')
    total_arret_penalty_match = str(row['penalty  buts / mj']/100)
    return [prenom,nom,matchs,but_total,occasion_total,pourcentage_buts,total_but_match,arrets_tirs,
    tirs_total,pourcentage_arrets_tirs,total_arret_tirs_match,arrets_penalty,penalty_total,pourcentage_arrets_penalty,total_arret_penalty_match]

In [116]:
df_result = stats_joueurs.apply(reformate_stats_joueurs,axis=1,result_type='expand')

In [117]:
df_result.columns = ['prenom','nom','match_joues','total_buts','total_occasions','pourcentage_buts','buts/match','buts_tirs',
    'total_tirs','pourcentage_buts_tirs','buts_tirs/match','buts_penalty','total_penalty','pourcentage_buts_penalty','buts_penalty/match']

In [119]:
df_result.tail(20)

,prenom,nom,match_joues,total_buts,total_occasions,pourcentage_buts,buts/match,buts_tirs,total_tirs,pourcentage_buts_tirs,buts_tirs/match,buts_penalty,total_penalty,pourcentage_buts_penalty,buts_penalty/match
250,Gabriel,BODINIER,13,0,0,0.00,0.0,0,0,0.00,0.0,0,0,0.00,0.0
251,Maxime,DIOT,24,0,0,0.00,0.0,0,0,0.00,0.0,0,0,0.00,0.0
252,Hugo Miguel,BRANCA FREITAS,6,0,0,0.00,0.0,0,0,0.00,0.0,0,0,0.00,0.0
253,Darius,MAKARIA,26,0,0,0.00,0.0,0,0,0.00,0.0,0,0,0.00,0.0
254,Hugo,KRISZT,11,0,0,0.00,0.0,0,0,0.00,0.0,0,0,0.00,0.0
255,Julien,SALMON,14,0,0,0.00,0.0,0,0,0.00,0.0,0,0,0.00,0.0
256,Quentin,HUGOT,1,0,0,0.00,0.0,0,0,0.00,0.0,0,0,0.00,0.0
257,Oscar,DEMIOT,5,0,0,0.00,0.0,0,0,0.00,0.0,0,0,0.00,0.0
258,Nathan,RENAUD,3,0,0,0.00,0.0,0,0,0.00,0.0,0,0,0.00,0.0
259,Cory,LAURENT,5,0,0,0.00,0.0,0,0,0.00,0.0,0,0,0.00,0.0


In [55]:
YEAR = ['2012-13','2013-14','2014-15','2015-16','2016-17','2017-18','2018-19','2019-20','2020-21','2021-22','2022-23']
LIGUE = ['D1H','D2H']

In [ ]:
f'data/D2H/stats_gardiens_proligue_{YEAR[i-24]}.csv')

# Formating info

In [120]:
df = pd.read_csv('/Users/bpugnaire/Dev/github/handball-scraping/data/RAW/D1H/joueurs_lnh_2019-20.csv')

In [121]:
df

,Unnamed: 0,prenom_nom,poste,numero,equipe,photo_url,cohorte
0,0,Luc ABALO,Ailier Droit,19.0,Paris,https://www.lnh.fr/medias/sports_players/abalo...,PRO
1,1,Ayyoub ABDI,Arrière Droit,87.0,Toulouse,https://www.lnh.fr/medias/sports_players/abdi-...,PRO
2,2,Jean-jacques ACQUEVILLO,Arrière Gauche,27.0,Nîmes,https://www.lnh.fr/medias/sports_players/acque...,PRO
3,3,Titouan AFANOU GATINE,Arrière Droit,88.0,Chartres,https://www.lnh.fr/medias/sports_players/afano...,PRO
4,4,Benjamin AFGOUR,Pivot,33.0,Montpellier,https://www.lnh.fr/medias/sports_players/afgou...,PRO
...,...,...,...,...,...,...,...
367,2,Martial CAÏS,Ailier Droit,NaN,Saint-Raphaël,https://www.lnh.fr/medias/sports_players/silho...,RES
368,3,Quentin HULOT,Gardien,12.0,Dunkerque,https://www.lnh.fr/medias/sports_players/hulot...,RES
369,4,Dylan MALIGUE,Non défini,25.0,Istres,https://www.lnh.fr/medias/sports_players/malig...,RES
370,5,Enzo PESENTI ROSSI,Non défini,NaN,Tremblay,https://www.lnh.fr/medias/sports_players/pesen...,RES


In [122]:
def formate_prenom_nom(row):
    joueur = row['prenom_nom']
    joueur = upperify_accents(joueur)
    try:
        debut_nom = next(re.finditer('([A-ZÀ-Ú][A-ZÀ-Ú])', joueur)).start(0)
        nom = unidecode(joueur[debut_nom:]).upper()
        prenom = unidecode(joueur[:debut_nom-1])
    except:
        prenom = ''
        nom = ''
    return [prenom,nom]

In [125]:
df[['prenom','nom']] = df.apply(formate_prenom_nom,axis=1, result_type = 'expand')

In [128]:
df = df.drop(columns =['Unnamed: 0', 'prenom_nom'])

In [131]:
df = pd.read_csv('/Users/bpugnaire/Dev/github/handball-scraping/data/MERGED/joueurs_lnh.csv')


In [133]:
df['nom'].groupby([df.saison, df.ligue, df.cohorte]).agg('count')

saison   ligue  cohorte
2004-05  D1H    CF          72
                PRO        204
2005-06  D1H    CF          76
                PRO        224
                RES          1
                          ... 
2022-23  D1H    PRO        253
                RES         14
         D2H    CF          34
                PRO        235
                RES         53
Name: nom, Length: 74, dtype: int64